# Taxonomy examples
## Project: Taxonomy of urban bicycle network approaches

This notebook downloads selected bicycle and street network data sets and plots them.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2021-01-05  
Last modified: 2021-01-06

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
redownload = False # If True, will re-download data even if already there
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

### Functions

In [ ]:
%run -i functions.py

## Download data

In [ ]:
Gs = {}
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    Gs[placeid] = {}
    for parameterid, parameterinfo in tqdm(osmnxparameters.items(), desc = "Networks", leave = False):
        if redownload or not os.path.isfile(PATH["data"] + placeid + "_" + parameterid + "_edges.zip"):
            for i in range(0,10): # retry loop
                try:
                    Gs[placeid][parameterid] = ox.graph_from_point(
                                        center_point = (float(placeinfo["lat"]), float(placeinfo["lon"])), 
                                        dist = int(placeinfo["dist"]),
                                        network_type = parameterinfo['network_type'],
                                        custom_filter = (parameterinfo['custom_filter']),
                                        retain_all = parameterinfo['retain_all'],
                                        simplify = False
                                        )
                except ValueError:
                    Gs[placeid][parameterid] = nx.empty_graph(create_using = nx.MultiDiGraph)
                    break
                except: # osmnx throws a variety of connection/protocol/unboundlocal/etc. errors here
                    print("Connection problem. Retrying.")
                    time.sleep(0.1 * 2**i)
                    continue
                break

            ox_to_csv(Gs[placeid][parameterid], PATH["data"], placeid, parameterid)

## Plot networks

In [ ]:
%%capture
biketracks = ["bike_highwaycycleway", "bike_cyclewaytrack", "bike_bicycledesignated", "bike_designatedpath", "bike_cyclewayrighttrack", "bike_cyclewaylefttrack"]
Gs = {}
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    
    # Load networks
    Gs[placeid] = {}
    for parameterid, parameterinfo in osmnxparameters.items():
        Gs[placeid][parameterid] = csv_to_ig(PATH["data"], placeid, parameterid)
    # Plot
    fig = initplot()
    map_center = nxdraw(Gs[placeid]["carall"], "carall")
    nxdraw(Gs[placeid]["car30"], "car30", map_center)
    for fil in biketracks:
        nxdraw(Gs[placeid][fil], "bike_onstreet", map_center, nnids = list(set([v["id"] for v in Gs[placeid][fil].vs]).intersection(set([v["id"] for v in Gs[placeid]["carall"].vs]))))
    for fil in biketracks:
        nxdraw(Gs[placeid][fil], "bike_offstreet", map_center, nnids = list(set([v["id"] for v in Gs[placeid][fil].vs]).difference(set([v["id"] for v in Gs[placeid]["carall"].vs]))))
    nxdraw(Gs[placeid]["bike_cyclestreet"], "bike_sharedstreet", map_center)
    nxdraw(Gs[placeid]["bike_livingstreet"], "bike_sharedstreet", map_center)
    plt.savefig(PATH["plots"] + placeid + '.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()